## RSA

### Key Generation

In [16]:
p = 1613
q = 1949
e = 2 ^ 16 + 1
e = 65537

In [17]:
n = p * q
n

3143737

In [18]:
phi_n = (p-1) * (q-1)
phi_n

3140176

In [19]:
d = mod(1/e, phi_n)
d

1865601

### Decryption

In [20]:
n, e = 1173031, 65537
n, d = 1173031, 378449
c = 601090

In [21]:
m = mod(c ^ d, n)
m

888341

### Encryption

In [24]:
c = mod(m ^ e, n)
c

601090

### CRT - RSA

In [28]:
n = 396553
p = 541
q = 733
e = 17
phi_n = (q-1) * (p-1)

In [29]:
u = int(mod(1/p, q)) 
u

691

In [30]:
d = int(mod(1/e, phi_n))
d

302273

In [31]:
dp = int(mod(d, p-1))
dp

413

In [32]:
dq = int(mod(d, q-1))
dq

689

In [33]:
# Decrypt
c = 234040
cp = c
cq = c

mp = int(mod(cp^dp, p))
mq = int(mod(cq^dq, q))

m = int(mod(mp + (p * u * (mq - mp)), n))
m

332211

In [34]:
# Test Reencrypt
c_new = mod(m ^ e, n)
c_new

234040

## Factorization

### p-1 for RSA factorization

In [200]:
n = 635404704211 // 89
a = 225170

In [201]:
def calc_s(B):
    lcm_B = []
    for b in range(1, B+1):
        lcm_B.append(b)
    return lcm(lcm_B)

In [209]:
s = calc_s(16)
s.factor()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


2^4 * 3^2 * 5 * 7 * 11 * 13

In [203]:
b = int(mod(a ^ s, n))
b

1194511837

In [207]:
p = gcd(b - 1, n)
p
(p-1).factor()

2^3 * 3^5 * 5 * 7 * 11

In [208]:
q = n / p
(q-1).factor()

2 * 19 * 251

In [206]:
assert n == p * q

In [110]:
c = float(89^(1/4))
c

3.0714786556407327

### Pollard Rho for Factorization

In [112]:
n = 635404704211
# // 89
# 89 * 9539 * 748441

# B is the smoothness of the prime I am looking for
B = int(n ^ (1/4))

# Note: there could be a constant d
def next_p(p):
    c = 73
    return int(mod(p ^ 2 + c, n))

p0 = 426166815697
S = 1
s_1 = p0
s_2 = p0
for i in range(5):
    s_1 = next_p(s_1)
    s_2 = next_p(next_p((s_2)))
    diff = int(mod(s_2 - s_1, n))
    S = mod(S * diff, n)
p = gcd(S, n)
# check if p is prime
# if not, try with another c and d parameters for calculating the next step
# If you do not get any factors that are prime, then you increase the range B,
# if you get that gcd(s,n) = n, then you decrease B
print("S", S)
print("p", p)
q = n / int(p)
print("q", q)

S 604394567889
p 89
q 7139378699


In [113]:
5^4

625

## Coppersmith Method

Coppersmith' method works to find small roots of polynomials. In this case the polynomial is f(x) = (a + x)^5 - c \bmod nf(x)=(a+x) 
5
 −cmodn from the encryption equation c = m^3 \bmod nc=m 
3
 modn and the known top part aa of mm from the stereotyped message. The unknown part xx is less than X = 36^6= 2176782336X=36 
6
 =2176782336 which is small on the scale of nn which is about 36^{91}36 
91
 .
The method works by setting up a lattice, given by a matrix, so that each row is has xx as a root modulo nn. The top row is given by f(x)f(x), where the left-most columns corresponds to x^5x 
5
  and the rightmost column to x^0x 
0
 .
Because of a result of Howgrave-Graham on when the method succeeds and to balance the weights, the column of x^ix 
i
  is actually scaled by X^iX 
i
 , using the upper bound XX on xx. Hence, the first row matches the coefficients of
f(xX) = (a + xX)^5 = X^5x^5 + 5 X^4ax^4 + 10X^3a^2x^3 + 10X^2a^3x^2 + 5Xa^4x + a^5 - cf(xX)=(a+xX) 
5
 =X 
5
 x 
5
 +5X 
4
 ax 
4
 +10X 
3
 a 
2
 x 
3
 +10X 
2
 a 
3
 x 
2
 +5Xa 
4
 x+a 
5
 −c. The other rows are given by the coefficients of X^inx^iX 
i
 nx 
i
 .
The lattice is given by
M = [[X^5 + 5* X^4*a , 10*X^3*a^2 , 10*X^2*a^3 , 5*X*a^4, a^5 - c],
[0,X^4*n,0,0,0,0],
[0,0,X^3*n,0,0,0],
[0,0,0,X^2*n,0,0],
[0,0,0,0,X*n,0],
[0,0,0,0,0,n]]
The last 5 rows are identical to 0 modulo nn so satisfy the rule and ensure that each position can be reduced modulo n.
Using LLL we can obtain a basis with short vectors which implies small coefficients.
Howgrave-Graham gives conditions on the norm of the coefficient vector under which a root modulo nn is also a root over the integers. Combining this with LLL gives (apart from some smaller factors) that the method succeeds if $$\root[6](\det(M)) < n$$. Here \det(M) = n^5X^{15}det(M)=n 
5
 X 
15
 , hence the condition translates to X^{15}<nX 
15
 <n. In this case we have X = 36^6X=36 
6
  and 36^{90}<n36 
90
 <n, so the method should succeed.

### For RSA primes

In [29]:
# Case from slides
p = random_prime(2^512); q = random_prime(2^512)
n = p*q # nothing suspicious here
a = p - (p % 2^160) # partial info we learn

# we obtain a in the form of p = a + r
# 160 number of bits we don't know
X = 2^160

In [36]:
# Case from exam
r = (random_prime(2^32))
c = log(r, 2).n()
print(r, c)

a = 3141592653589793238462643383279502884197 * 10^11 
a
print(log(p1, 2).n())
X = 2^32
n = 85397342226735670654635508695465744950338290459323016319434988214268887865669909148767449445636761969

4062363857 31.9196723184308
167.747900873840


In [185]:
# EXAM
n = 67890055483009241509585928845990056768301355636549437937029809249749727319
p_top = 7975367974709495237422842361682067456
missing = 123 - 83
# 40 bits is less than 1/3 of 123
e = 2^16 + 1
X = 2^40
a = p_top
# p.bits()
len(p_top.bits())

123

Make sure a=7643764000000, X=2^r and n are defined, then check for p

In [186]:
M = matrix([[X^2, X*a, 0], [0, X, a], [0, 0, n]])

In [187]:
B = M.LLL()

In [188]:
Q = (B[0][0] * x^2 )/X^2 + (B[0][1] * x)/ X + B[0][2]

In [191]:
Q.roots(ring=ZZ)

[(21276691477, 1)]

In [196]:
q = n / (a + Q.roots(ring=ZZ)[0][0])

In [198]:
p = n / q
m = p * q
m == n
print(f"p {p}, q {q}")
mod(p, 2^40)

p 7975367974709495237422842382958758933, q 8512466847710829752670576779133440443


21276691477

### For Messages

In [186]:
a = Integer('The ticket code for entry to the event is 000000' , 36)
a

4065148191661314021859159447510046461975168206921387422466048

In [187]:
m.str(36)

'theticketcodeforentrytotheeventis000000'

In [188]:
n = 22025482441888726596151828037580210653496214995318500244177792057815419084860452102794997828952857497081479129671501030354465069182775254612751
e = 5

In [189]:
c = 18026784157801727131455776136196465618870787523053645052562578424425084803098429518969764363704209533679484842464484114144371404287037590112297

In [193]:
# If it does not work, try with 'zzzzzz'
X = Integer('ffffff',36)
# (x + a)^5 - c
# where e = 5
# for top row use https://www.symbolab.com/solver/polynomial-equation-calculator/%5Cleft(x%2B%20a%5Cright)%5E%7B5%7D-c?or=input
M = matrix([
    [X^5, 5 * X^4 * a, 10 * X^3 * a^2, 10 * X^2 * a^3, 5 * X * a^4, a^5 - c],
    [0,   n * X^4,     0,              0,              0,           0], 
    [0,   0,           n * X^3,        0,              0,           0],
    [0,   0,           0,              n * X^2,        0,           0],
    [0,   0,           0,              0,              n * X,       0],
    [0,   0,           0,              0,              0,           n]
])

In [194]:
B = M.LLL()
Q = (B[0][0] * x^5 / X^5) + (B[0][1] * x^4 / X^4) + (B[0][2] * x^3 / X^3) + (B[0][3] * x^2 / X^2) + (B[0][4] * x / X) + B[0][5]

In [195]:
Q.roots(ring=ZZ)[0][0].str(base=36)

'1497tt'

## Diffie Hellman

In [169]:
# https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange
p = 1000003
g = 2
a = 322470
hB = 55013

In [170]:
hA = mod(g^a, p)
hA

514229

In [171]:
H = mod(hB^a, p)
H

553408

## Discrete Log

### Baby Step Gian Step

In [62]:
def BSGS(N, g, ga):
    
    
    m = floor(sqrt(N))
    
    bs_dict = {}
    bs_dict[1] = 0
    bs_dict[g] = 1 
    
    # Correct
    # Baby step
    g_temp = g
    for i in range(1, m):
        g_temp = mod(g ^ i, N)
        bs_dict[g_temp] = i
    
    # Correct
    # Giant step
    # print(f"m {m}")
    R = mod(g ^ (-m), N)
    j = 0
    Q = ga
    while Q not in bs_dict:
        j += 1
        Q = mod(Q * R, N)
        
    i = bs_dict[Q]
    
    return mod(j * m + i, N)

### Pollig-Hellman Attack

In [40]:
# order N in array of tuples in the form (prime, exponent)
# N_list = [(2, 4), (7, 2), (37, 1)] 
# N = (2 ** 4) * (7 ** 2) * 37 + 1
# N is prime

# http://math.ucdenver.edu/~wcherowi/courses/m5410/phexam.html

def pohling_hellman(g: int, Q: int, N: int, N_list: list[(int, int)]):
    crm_list = []
    
    for i in range(len(N_list)):
        
        pi, ei = N_list[i]
        Qi = Q
        a_list = []
        
        for j in range(1, ei + 1):    
            
            e = int(((N-1) / pi^j))
            aij = mod(Qi ^ e, N)
            
            #BSGS
            # gij = mod(g ^ e, N)
            # print(f"pi {pi}, e {e}, gij {gij}, aij {aij}")
            # bs = BSGS(pi, gij, aij)
            # a_list.append(bs)
            
            # This is ok for small primes
            # Brute force
            for k in range(pi + 1):
                e = k * (N-1) / pi
                #print(f"e {e}") 
                if mod((g ^  e), N) == aij:
                    a_list.append(k)
                    inv_g = mod(g ^ (-k * pi^(j -1)), N)
                    Qi = mod(Qi * inv_g, N)
                    break

        x = 0
        j = 0
        print(f"list a for {N_list[i][0]}: {a_list}")
        for a in a_list:
            x += a * (pi ^ j)
            j += 1
        crm_list.append(x)
        
    factors_list = [ p^e for p,e in N_list]
    a = crt(crm_list, factors_list)
    return a

In [41]:
# Example from http://math.ucdenver.edu/~wcherowi/courses/m5410/phexam.html
N = 8101 # prime
N_list = [(2, 2), (3, 4), (5, 2)]
g = 6
B = 7531

exp = pohling_hellman(g, B, N, N_list)
print("exp", exp)
factor(N-1)


list a for 2: [1, 0]
list a for 3: [2, 0, 2, 1]
list a for 5: [4, 2]
exp 6689


2^2 * 3^4 * 5^2

In [42]:
# example from exam 144078 exercise 2
g = 10
N = 22916251
B = 14876484
N_list = [(2, 1), (3, 3), (5, 4), (7, 1), (97, 1)]

exp = pohling_hellman(g, B, N, N_list)
print("exp", exp)

list a for 2: [0]
list a for 3: [2, 1, 1]
list a for 5: [2, 4, 2, 2]
list a for 7: [2]
list a for 97: [33]
exp 16236572


In [43]:
# Verify your result
hB = mod(g ^ 16236572, N)
print(hB)
hB == B

14876484


True

### Pollard Rho & Floyds

In [51]:
PN = 22916251
e = (PN-1)/97
H = int(mod(B^e, PN))

g = 10
G = int(mod(g^e, PN))

N = 97
# Pollard rho
# steps (ri, ti)
steps = [(10, 84), (95, 1), (15, 28), (1, 84), (14, 67)]
s_list = []
for step in steps:
    ri, ti = step
    # print(ri, ti)
    si = int(mod(G^ri * H^ti, PN))
    s_list.append(si)
s_list    

[16167847, 7631738, 16167847, 3553851, 21514752]

In [52]:
G, H, N

(2429001, 3035691, 97)

In [57]:
def next_step(x, r, t):
    i = mod(x, 5)
    x = int(mod(s_list[i] * x, PN))
    ri, ti = steps[i]
    r += ri
    t += ti
    return x, r, t
        
def pollard_rho(G, H, N):
    s = 6
    x = int(mod(G^s, N))
    s_slow = s_fast = x
    r_slow = r_fast = s
    t_slow = t_fast = 0
    assert s_fast == int(mod(G^r_fast * H^t_fast , N))
    for i in range(1, 20):
        s_slow, r_slow, t_slow = next_step(s_slow, r_slow, t_slow)
        s_fast, r_fast, t_fast = next_step(*next_step(s_fast, r_fast, t_fast))
        assert s_fast == int(mod(G^r_fast * H^t_fast , N))
        print(f"{i} slow is {s_slow}, fast is {s_fast}")
        if (s_slow == s_fast):
            print("FOUND IT!!!")
            return mod((r_fast - r_slow)/(t_slow - t_fast), 97)
    return a

In [56]:
rho = pollard_rho(G, H, PN)
rho

1 slow is 20809813, fast is 16172679
2 slow is 16172679, fast is 8430602
3 slow is 12093259, fast is 8430602
4 slow is 8430602, fast is 8430602
FOUND EM!!!


33

In [61]:
# assertion
int(mod(int(G)^int(rho), PN)) == H

True

In [106]:
# Example from exam /80764 #2 Discrete Log
N_list = [(2, 4), (7, 2), (37, 1)] 
N = (2 ** 4) * (7 ** 2) * 37
P = 29009
g = 11
Q = 12542

b = pohling_hellman(g, Q, P, N_list)

print(b)
mod(g ^ b, P)

list a for 2: [0, 0, 1, 0]
list a for 7: [4, 4]
list a for 37: [34]
3364


12542

### Verify Results

## Elliptic Curves

### Edwards, Twisted Edwards, Weierstrass, Montgomery

In [114]:
## Edwards Curve
## Form: x^2 + y^2 = 1 + d*x^2*y^2, with d < 0
## Usually computed mod n
## Point addition

## In Twisted Edwards curve the a is set to something different than 1

def edwards_point_addition(p1: (int, int), p2: (int, int), d: int, n: int, a = 1) -> (int, int):
    xa, ya = p1
    xb, yb = p2
    den = d * xa * ya * xb * yb
    xc = ((xa * yb) + (xb * ya))/(1 + den)
    xc = int(mod(xc, n))
    yc = ((ya * yb) - (a * xa * xb))/(1 - den)
    yc = int(mod(yc, n))
    return (xc, yc)

def edwards_point_doubling(p1: (int, int), d: int, n: int, a=1) -> (int, int):
    pass

def point_check(p: (int, int), d: int, n: int, a=1) -> bool:
    x, y = p
    a = mod(a * x^2 + y^2, n)
    b = mod(1 + d * x^2 * y^2 , n)
    return a == b
    
d = -5
n = 13
p = (6, 3)
# print(point_check(p, d, n))


# Montgomery and Weistrass implement with Sage



# Weierstrass
# y^2 = x^3 + c4 * x + c6
c4, c6 = 3, 6
K = GF(43) # Field 
W = EllipticCurve(K, [c4, c6])

# Montgomery
# B *y^2 = x^3 + A * x^2 + x
# for B != 0, A != 2, -2
# Montgomery implemented in homework

In [115]:
n = 2^31 - 1
a = n - 1
d = 1990442015
p = (486722789, 1078198773)
p2 = edwards_point_addition(p, p, d, n, a)

# e2 = edwards_point_addition(p, e, d, n, a)
# e2
point_check(p, d, n)

False

In [117]:
A = mod(2* (a+d) / (a-d), n)
B = mod(4 / (a-d), n)
print(f"A {A}, B {B}")

A 528003815, B 1619479830


In [123]:
def to_mont(p):
    # To montgomery
    x, y = p
    u = (1+y) / (1-y)
    v = u / x
    return (mod(u, n), mod(v, n))

In [127]:
Q = to_mont(p)
Q2 = to_mont(p2)
print(f"Q {Q}, Q2 {Q2}")

Q (1986357151, 1124510096), Q2 (1234293321, 1843043349)


In [128]:
# Montgomery
# B * v^2 = u^3 + A * u^2 + u
v = 1124510096
left = B * v^2
left = int(mod(left , n))

u = 1986357151
right = u^3 + A*u^2 + u
right = int(mod(right , n))

left == right


True

In [129]:
def doubling_point(p1, A, B):
    """
    montgomery doubling
    """
    x1,y1 = p1
    
    x3 = B*(3*x1^2+2*A*x1+1)^2 / (2*B*y1)^2 - A - x1 -x1
    y3 = ((2*x1+x1+A)*(3*x1^2+2*A*x1+1) / (2*B*y1)) - ((B*(3*x1^2+2*A*x1+1)^3) / (2*B*y1)^3 ) - y1
    
    return (mod(x3, n), mod(y3, n))

In [138]:
def mont_double_slope(Q, A, B):
    u, v = Q
    num = 3*u^2 + 2*A*u +1
    div = 2 * B * v
    slope = num / div
    return slope

In [139]:
slo = mont_double_slope(Q, A, B)
slo

6983345

In [130]:
Q3 = doubling_point(Q, A, B)
Q3

(1234293321, 1843043349)

### Birational Equivalence

### Discrete Log Attacks:

#### Baby Step Giant Step

#### Pollard Rho

#### Pollig Hellman

In [167]:
p = 424243
K = GF(p) # Field 
A = 518
c1 = 0
c2 = A
c3 = 0
c4 = 1
c6 = 0

W = EllipticCurve(K, [c1, c2, c3, c4, c6])

P = W(53880, 381693)
Q = W(245898,70893)

n = 2^5 * 5
assert n == P.order()

N_list = [(2, 5), (5, 1)]

In [152]:
def BSGS(N, P, Pa):

    m = floor(sqrt(N))
    
    bs_dict = {}
    bs_dict[P.curve()(0,1,0)] = 0
    bs_dict[P] = 1 
    
    # Baby step
    p_temp = P
    for i in range(2, m):
        # EC addition
        p_temp = P + p_temp
        bs_dict[p_temp] = i
        
    # Giant step
    # EC addition
    R = P + p_temp
    j = 0
    Q = Pa
    while Q not in bs_dict:
        j += 1
        # EC addition
        Q = Q + R
        
    i = bs_dict[Q]
    
    return mod(i - j * m, N)

In [165]:
def pohling_hellman(P, Q, N, N_list):
    crm_list = []
    
    for i in range(len(N_list)):
        Qi = Q
        a_list = [0]
        pi, ei = N_list[i]
        ni = int(N / pi)
        Ri = ni * P
        for j in range(0, ei):
            if (j != 0):
                ni = int(N / (pi ** (j + 1)))
            tmp = int(a_list[j] * (pi^(j-1)))
            Qi = Qi - (tmp *  P)
            Si = ni * Qi
            # Solve DLP with BSGS
            aij = int(BSGS(pi, Ri, Si))
            a_list.append(aij)

            
        print(f"a_list for {pi}: {a_list}")
        ai = 0
        for idx,aij in enumerate(a_list[1:]):
            ai += aij*(pi^idx)
        # Mod
        crm_list.append(ai)
                
    # Solve CRT
    factors_list = [ p^e for p,e in N_list]
    a = crt(crm_list, factors_list)
    
    return a

In [166]:
pohling_hellman(P, Q, n, N_list)

a_list for 2: [0, 1, 0, 1, 0, 1]
a_list for 5: [0, 2]


117

In [155]:
R = P * 117
print(f"R {R}, Q {Q}")

R (245898 : 70893 : 1), Q (245898 : 70893 : 1)


### Signatures with ECDSA:

## MACs

## Hash functions

## Signatures

## Symmetric crypto

## Questions

### Exam 1 - 3 b

Describe in your own words how and why Pollard's rho method for factorization works and what the
expected runtime is depending on the factors of n.

Use this to explain why you managed to factor n in part a) and explain why the p-1 method would
likely not have succeded for factoring n when used with B1 = 10, i.e., s =lcm(1, 2, 3, ... , 10).

### Exam 1 - 4 a

Explain how and why you can use Coppersmith' method to compute the missing part of m from c
given the above information, i.e., the known part of m and that the event codes have 6 alphanumeric
characters as well as n and e.

### Exam 2 - 2 e

Explain why the p − 1 method was successful in factoring this n.

### Exam 4 - 5

Is it Coppersmith?

### Exam 4 - 2 a

Give a short explanation of how and why the Pohlig-Hellman attack works and
why the steps below allow you to compute all of b (modulo p − 1).

In [1]:
2^32

4294967296

In [2]:
2^32 - 2^31

2147483648

In [49]:
is_prime(3373)

True